In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import talib as ta
import yfinance as yf

sns.set()

In [3]:
def fwd_return(Px,L):
    fwd = Px.shift(-L)
    ret = 10000*np.log(fwd/Px)
    ret.name = 'F'+str(L)+'ret'
    return ret

def ta_pattern_fit(hist,L,candle_pattern):
    expression = 'ta.'+candle_pattern+'(hist.Open,hist.High,hist.Low,hist.Close)'
    patt = eval(expression)
    patt.name = candle_pattern
    Fret = fwd_return(hist['Close'],L)
    df = pd.concat([patt,Fret],axis = 1).dropna()
    
    avg = df.groupby(candle_pattern)[Fret.name].agg({'mean','std','count'})
    
    Npos = df.groupby(candle_pattern)[Fret.name].apply(lambda x: x[x>0].count())
    Npos.name = 'Npos'
    
    AW = df.groupby(candle_pattern)[Fret.name].apply(lambda x: x[x>0].mean())
    AL = df.groupby(candle_pattern)[Fret.name].apply(lambda x: x[x<0].mean())
    
    AW.name = 'AW'
    AL.name = 'AL'
    
    summary = pd.concat([avg,Npos,AW,AL],axis = 1)
    summary['hit_ratio'] = summary['Npos']/summary['count']
    
    return summary
    
    

In [6]:
ticker = yf.Ticker("GBPUSD=X")
hist = ticker.history(period="max")

In [9]:
L = 3
candle_pattern = 'CDLRICKSHAWMAN'

In [10]:
Px = hist['Close']
summary = ta_pattern_fit(hist.loc['2020'],L,candle_pattern)

display(summary)

,mean,count,std,Npos,AW,AL,hit_ratio
CDLRICKSHAWMAN,,,,,,,
0,-6.259743,165,145.319425,77,101.305099,-100.378981,0.466667
100,18.389806,94,111.478281,53,87.740993,-71.259289,0.563830
